In [2]:
from pathlib import Path
from datatree import DataTree
import dask

import pandas as pd
import valenspy as vp
import xarray as xr

ERA5 Land dataset zonder time bnds

In [3]:
# Function to preprocess files and remove 'time_bnds'
def preprocess(ds):
    return ds.drop_vars('time_bnds', errors='ignore')

# Load ERA 5 land data with preprocessing to drop 'time_bnds'
era5_land_files = list(Path("/mnt/HDS_BREGILABEPOC/BREGILABEPOC/bertvs/era5-land/belgium/daily").rglob("*.nc"))
era5_land_ds = xr.open_mfdataset(era5_land_files, chunks="auto", preprocess=preprocess)

In [4]:
from valenspy.inputconverter_functions import EOBS_to_CF, ERA5_to_CF, ERA5Land_to_CF
ic = vp.InputConverter(ERA5Land_to_CF)


<xarray.Dataset> Size: 7GB
Dimensions:    (time: 52596, longitude: 41, latitude: 25)
Coordinates:
  * time       (time) datetime64[ns] 421kB 1950-01-01T11:00:00 ... 2021-12-31...
  * longitude  (longitude) float32 164B 2.0 2.125 2.25 2.375 ... 6.75 6.875 7.0
  * latitude   (latitude) float32 100B 52.0 51.88 51.75 ... 49.25 49.12 49.0
Data variables: (12/18)
    d2m        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    e          (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    pev        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    ro         (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    sde        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    sf         (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    ...         ...
    ssrd       (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    swvl1      (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    t2m        (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    tp         (time, latitude, longitude) float64 431MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 216MB dask.array<chunksize=(731, 25, 41), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 216MB dask.array<chunksize=(730, 25, 41), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.9 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Mon Sep 05 05:06:06 2022: cdo -z zip daymin /mnt/HDS_BREGIL...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.9 (https://mpimet.mpg.de...

In [70]:
#Var options = 10m_u_component_of_wind  2m_temperature         runoff            snowfall                     surface_pressure            surface_solar_radiation_downwards
#10m_v_component_of_wind  evaporation            skin_temperature  surface_latent_heat_flux     surface_runoff              total_precipitation
#2m_dewpoint_temperature  potential_evaporation  snow_depth        surface_net_solar_radiation  surface_sensible_heat_flux  volumetric_soil_water_layer_1

In [71]:
var="total_precipitation"
var_list = list(Path(f"/mnt/HDS_BREGILABEPOC/BREGILABEPOC/bertvs/era5-land/belgium/daily/{var}").glob("*max*.nc"))
var_ds = xr.open_mfdataset(var_list, chunks="auto")
var_ds 

<xarray.Dataset> Size: 216MB
Dimensions:    (time: 26298, bnds: 2, latitude: 25, longitude: 41)
Coordinates:
  * time       (time) datetime64[ns] 210kB 1950-01-01T11:00:00 ... 2021-12-31...
  * longitude  (longitude) float32 164B 2.0 2.125 2.25 2.375 ... 6.75 6.875 7.0
  * latitude   (latitude) float32 100B 52.0 51.88 51.75 ... 49.25 49.12 49.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 421kB dask.array<chunksize=(365, 2), meta=np.ndarray>
    tp         (time, latitude, longitude) float64 216MB dask.array<chunksize=(365, 25, 41), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.9 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Mon Sep 05 05:32:53 2022: cdo -z zip daymax /scratch/bertvs...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.9 (https://mpimet.mpg.de...

In [72]:
#Merge the two datasets Something is wrong with the time bands of the two datasets... 
era5_land_ds = xr.merge([era5_land_ds, var_ds])

MergeError: conflicting values for variable 'time_bnds' on objects to be combined. You can skip this check by specifying compat='override'.